In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as matdates
from datetime import datetime as dt
from datetime import date, timedelta
import time
import yfinance as yf
import random
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
stock_list = { 'SPY' : '미국 대형주 / SPY',
              'IWD' : '미국 대형가치주 / IWD',
              'IWM' : '미국 소형주 / IWM',
              'IWN' : '미국 소형가치주 / IWN',
              'MTUM' : '미국 모멘텀주 / MTUM',
              'EFA' : '미국 외 선진국 주식 / EFA',
              'TLT' : '미국 장기채권 / TLT',
              'IEF' : '미국 중기채권 / IEF',
              'LQD' : '미국 회사채권 / LQD',
              'DBC' : '원자재 / DBC',
              'VNQ' : '미국 부동산 / VNQ',
              'BWX' : '국제 국채 / BWX', 
              'GLD' : '금 / GLD'} 

In [4]:
def to_index(x): # 인덱스화
    for i in range(1,len(x)):
        x.iloc[i] = (x.iloc[i]/x.iloc[0])*100
    x.iloc[0] = 100
def get_return(x):  #기간 동안의 첫번째 종가와 마지막 시가로 수익률 계산
    global stock_return
    rate_of_return = round((x.iloc[-1]-100),2)
    stock_return.append(rate_of_return)

In [5]:
start_time = time.time()

period = [4,12,24,48]
df = pd.DataFrame({'1 Month' : [], '3 Month' : [], '6 Month' : [], '12 Month' : []})

end = dt.now()

cols = []
for i in period:
    start = end - timedelta(weeks=i)
    stock_return = []
    stock_category = []
    ten_mon_avg = []
    current_value = []
    for code, category in zip(stock_list.keys(), stock_list.values()):
        ten = yf.download(code, end - timedelta(weeks=40))['Adj Close']
        ten_mon = ten.mean()
        ten_mon_avg.append(ten_mon)
        current = yf.download(code, end - timedelta(days=3))['Adj Close']
        current_value.append(current[-1])
        code = yf.download(code, start)['Adj Close']
        to_index(code)
        get_return(code)
        stock_category.append(category)
    df[str(i//4)+' Month'] = stock_return
    cols.append(str(i//4)+' Month')
df['Average'] = df[cols].mean(axis=1).round(2)
df['Price Average for Ten Months'] = ten_mon_avg
df['Last Price'] = current_value
df['Category / Ticker'] = stock_category
df = df.sort_values('Average', ascending=False).reset_index(drop=True)
df.insert(5, 'Top Rank', range(1, 1 + len(df)))
df.loc[df['Last Price'] > df['Price Average for Ten Months'], 'Buy'] = True
df.loc[3:, 'Buy'] = False

print("--- %s seconds ---" % (time.time() - start_time))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [6]:
df

,1 Month,3 Month,6 Month,12 Month,Average,Top Rank,Price Average for Ten Months,Last Price,Category / Ticker,Buy
0,4.09,9.18,13.15,13.29,9.93,1,394.783573,429.899994,미국 대형주 / SPY,True
1,-1.18,6.74,10.37,19.21,8.78,2,65.918352,71.519997,미국 외 선진국 주식 / EFA,True
2,6.13,7.37,7.51,9.30,7.58,3,178.027667,185.029999,미국 소형주 / IWM,True
3,6.71,5.66,2.38,4.61,4.84,4,138.815817,140.169998,미국 소형가치주 / IWN,False
4,2.12,5.07,2.30,7.50,4.25,5,149.882687,153.630005,미국 대형가치주 / IWD,False
...,...,...,...,...,...,...,...,...,...,...
8,-1.24,2.84,-5.13,2.99,-0.14,9,140.397835,137.979996,미국 모멘텀주 / MTUM,False
9,0.96,2.50,0.97,-6.83,-0.60,10,82.968462,82.680000,미국 부동산 / VNQ,False
10,-2.10,-1.28,2.15,-3.07,-1.08,11,96.452188,96.940002,미국 중기채권 / IEF,False
11,-0.97,-3.02,3.08,-8.57,-2.37,12,102.151428,101.919998,미국 장기채권 / TLT,False


In [7]:
start_time = time.time()

period = [4,12,24,48]
df = pd.DataFrame({'1 Month' : [], '3 Month' : [], '6 Month' : [], '12 Month' : []})

end = dt.now()

cols = []
for i in period:
    start = end - timedelta(weeks=i)
    stock_return = []
    stock_category = []
    ten_mon_avg = []
    current_value = []
    for code, category in zip(stock_list.keys(), stock_list.values()):
        ten = yf.download(code, end - timedelta(weeks=40))['Close']
        ten_mon = ten.mean()
        ten_mon_avg.append(ten_mon)
        current = yf.download(code, end - timedelta(days=3))['Close']
        current_value.append(current[-1])
        code = yf.download(code, start)['Close']
        to_index(code)
        get_return(code)
        stock_category.append(category)
    df[str(i//4)+' Month'] = stock_return
    cols.append(str(i//4)+' Month')
df['Average'] = df[cols].mean(axis=1).round(2)
df['Price Average for Ten Months'] = ten_mon_avg
df['Last Price'] = current_value
df['Category / Ticker'] = stock_category
df = df.sort_values('Average', ascending=False).reset_index(drop=True)
df.insert(5, 'Top Rank', range(1, 1 + len(df)))
df.loc[df['Last Price'] > df['Price Average for Ten Months'], 'Buy'] = True
df.loc[3:, 'Buy'] = False

print("--- %s seconds ---" % (time.time() - start_time))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [8]:
print(df.to_string())

    1 Month  3 Month  6 Month  12 Month  Average  Top Rank  Price Average for Ten Months  Last Price  Category / Ticker    Buy
0      4.09     9.18    12.72     11.89     9.47         1                    396.534219  429.899994       미국 대형주 / SPY   True
1      5.83     6.68     6.82      7.54     6.72         2                    179.346197  185.029999       미국 소형주 / IWM   True
2     -2.96     4.82     8.38     16.60     6.71         3                     67.193906   71.519997  미국 외 선진국 주식 / EFA   True
3     -2.76    -0.98     7.93     12.77     4.24         4                    171.386979  182.039993            금 / GLD  False
4      6.23     4.63     1.39      2.21     3.62         5                    140.367552  140.169998     미국 소형가치주 / IWN  False
5      1.70     4.10     1.35      5.13     3.07         6                    151.455261  153.630005     미국 대형가치주 / IWD  False
6     -1.48    -1.78     1.26     -0.27    -0.57         7                     22.328542   22.580000        국제 